In [1]:
import socketio
import eventlet
import numpy as np
from flask import Flask
from keras.models import load_model
import base64
from io import BytesIO
from PIL import Image
import cv2

In [2]:
sio = socketio.Server()


In [3]:
app = Flask(__name__) #'__main__'
speed_limit = 10
def img_preprocess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [4]:
@sio.on('telemetry')
def telemetry(sid, data):
    speed = float(data['speed'])
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
    image = img_preprocess(image)
    image = np.array([image])
    steering_angle = float(model.predict(image))
    throttle = 1.0 - speed/speed_limit
    print('{} {} {}'.format(steering_angle, throttle, speed))
    send_control(steering_angle, throttle)

In [5]:
@sio.on('connect')
def connect(sid, environ):
    print('Connected')
    send_control(0, 0)
 
def send_control(steering_angle, throttle):
    sio.emit('steer', data = {
        'steering_angle': steering_angle.__str__(),
        'throttle': throttle.__str__()
    })
 

In [6]:
from keras.losses import mean_squared_error as mse  

if __name__ == '__main__':
    model =load_model('model.h5', custom_objects={'mse': mse})


    app = socketio.Middleware(sio, app)
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(21908) wsgi starting up on http://0.0.0.0:4567
(21908) accepted ('127.0.0.1', 60563)


Connected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step
0.1446531116962433 0.9562 0.438
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
0.1446531116962433 0.9562 0.438
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
0.1446531116962433 0.9562 0.438
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
0.1438351422548294 0.9762 0.238
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
0.14349254965782166 0.8598 1.402
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
0.143593892455101 0.6892199999999999 3.1078
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
0.1437000036239624 0.49134999999999995 5.0865
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
0.14212574064731598 0.20572000000000001 7.9428
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
0.14250428974628448 -0.01753999999999989 10.1754
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
0.145548015832901 -0.20198000000000005 12.0198
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
0.14295083284378052 -0.2873300000000001 12.8733
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
0.14786779880523682 -0.2941100000000001 12.9411
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
0.145

127.0.0.1 - - [29/Oct/2024 18:09:56] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 13.585808
wsgi exiting
(21908) wsgi exited, is_accepting=True
